## tf-idf calculations

In [22]:
from tqdm import tqdm
tqdm.pandas()
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer

In [8]:
wpqs = pd.read_csv('cleaned.csv')
wpqs['cleanedQuestion'].fillna('', inplace=True)
wpqs['topic'].fillna('', inplace=True)

In [9]:
data = wpqs[['cleanedQuestion', 'topic']]

In [10]:
corpus = data.cleanedQuestion.tolist()

Instantiate and transform the vectorizer. 

In [14]:
tf_idf_vect = TfidfVectorizer()
X_train_tf_idf = tf_idf_vect.fit_transform(corpus)


/home/ben/miniconda3/envs/lan/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [15]:
terms = tf_idf_vect.get_feature_names_out()

In [18]:
cvec = CountVectorizer(stop_words = 'english', min_df = 3, max_df=0.5, ngram_range=(1,2))

In [19]:
sf = cvec.fit_transform(corpus)

In [23]:
transformer = TfidfTransformer()
transformed_weights = transformer.fit_transform(sf)
weights = np.asarray(transformed_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': cvec.get_feature_names(), 'weight': weights})

/home/ben/miniconda3/envs/lan/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [27]:
weights_df.sort_values('weight', ascending=False).head(10)

,term,weight
71274,department,0.017528
25769,assessment,0.013007
232599,steps,0.011686
250244,uk,0.010810
41257,care,0.010371
175065,plans,0.009816
227174,social,0.009626
239892,taking,0.009532
108123,government,0.009068
227185,social care,0.008786


## Gensim tf-idf

In [35]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /home/ben/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/ben/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

In [33]:
stemmer = SnowballStemmer

### Import the data

In [29]:
# Prepare our text

data_text = wpqs[['cleanedQuestion']]
data_text['index'] = data_text.index
documents = data_text

/tmp/ipykernel_76524/959812244.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_text['index'] = data_text.index


In [30]:
print(len(documents))
print(documents[:5])

385107
                                     cleanedQuestion  index
0  how much has been paid through working tax cre...      0
1  what the value was of the average claim for ta...      1
2  what the total value was of tax credits paid t...      2
3  how many self-employed people claimed (a) chil...      3
4  how many families in (a) york central constitu...      4


### Preprocessing

In [31]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [36]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['if', 'he', 'will', 'meet', 'the', 'hon.', 'member', 'for', 'middlesbrough', 'south', 'and', 'east', 'cleveland', 'to', 'discuss', 'the', 'proposed', 'closure', 'of', '(a)', 'minor', 'injuries', 'units', 'at', 'guisborough', 'and', 'east', 'cleveland', 'hospitals', '(b)', 'skelton', 'medical', 'centre', '(c)', 'p']


 tokenized and lemmatized document: 


TypeError: stem() missing 1 required positional argument: 'token'

In [37]:
doc_sample

'if he will meet the hon. member for middlesbrough south and east cleveland to discuss the proposed closure of (a) minor injuries units at guisborough and east cleveland hospitals (b) skelton medical centre (c) p'